In [76]:
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format
import glob
import os
import json
from simple_salesforce import Salesforce, format_soql, SFType, SalesforceLogin
from pprint import pprint
import datetime


In [77]:
with open("login.json", "r") as login_file:
    creds = json.load(login_file)

In [78]:
 #Login to SF
username=creds['login']['username']
password=creds['login']['password']
security_token=creds['login']['token']

session_id, instance = SalesforceLogin(
    username=username, password=password, security_token=security_token)

sf = Salesforce(instance=instance, session_id=session_id)

Here I am reading in the opportunity information from a csv file and convert it to the right Salesforce format, it is called "new_orders"

In [75]:
new_orders = new_orders.rename(columns={'Order quantity': 'Quantity', 'Unit price(in foreign currency)': 'UnitPrice', 'Order number':'Customer_PO__c', 
'Order date': 'Customer_PO_DATE__c', 'Delivery date1': 'ServiceDate'})

In [49]:
# changing to Salesforce datetime format
def change_po_date_SF(row):
        try:
            return row['Customer_PO_DATE__c'].isoformat() +'Z'
        except:
            return "UNSPECIFIED"        
        
new_orders['Customer_PO_DATE__c'] = new_orders.apply(change_po_date_SF,axis=1) 


# changing to Salesforce datetime format
def change_delivery_date_SF(row):
        try:
            return row['ServiceDate'].isoformat() +'Z'
        except:
            return "UNSPECIFIED"        
        
new_orders['ServiceDate'] = new_orders.apply(change_delivery_date_SF,axis=1) 

In [54]:
today = datetime.datetime.now()

In [ ]:
# Create an opportunity
# Enter opportunity name! 

new_opportunity = SFType('Opportunity', session_id, instance)
data = {
    'Name': 'some_name', 
    'StageName' :   'Closed Won', 
    'CloseDate' : (today).isoformat() +'Z',    #+ datetime.timedelta(days=45)).isoformat() + 'Z'  in the opportunity called "Decision date"
    'Segment__c' : 'Services', 
    'Vertical__c' : 'Spare parts',
    'NextStep' : 'Delivery',
    'AccountId' : '0010Y000xxxxxx', 
    'Probability' : '100',
    'Pricebook2Id' : '01s0Y00000xxxx'  
}
response = new_opportunity.create(data)
response


In [56]:
new_orders_to_SF['OpportunityId']=response['id']

<ipython-input-56-0f1983947875>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_orders_to_SF['OpportunityId']=response['id']


In [58]:
# putting everything into json format
to_sf_js = new_orders_to_SF.to_json(orient='records')
json_dicti = json.loads(to_sf_js)

In [ ]:
for i in json_dicti:
    new_opportunity_line_item = SFType('OpportunityLineItem', session_id, instance)
    print (i)
    response = new_opportunity_line_item.create(i)
    print (response)